In [ ]:
# import necessary libraries
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
import tensorflow as tf
from keras import layers
from keras.datasets import fashion_mnist
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Embedding
from keras.layers import BatchNormalization, Activation
from keras import initializers
from keras.initializers import RandomNormal
from keras.optimizers import Adam, RMSprop, SGD
from matplotlib import pyplot
import numpy as np
from math import sqrt
import os
from matplotlib import pyplot as plt
import cv2 as cv
from os import listdir
from numpy import asarray
from numpy import savez_compressed
from PIL import Image

# define a function to generate random points in the latent space
def generate_latent_points(latent_dim, n_samples):
    z_input = np.random.uniform(-1, 1, size=(n_samples , latent_dim))
    return z_input

# define a function to generate fake samples with the generator model
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict(z_input)  
    # create class labels
    y = zeros((n_samples, 1))
    return images, y

# define a function to generate real samples from the Fashion-MNIST dataset
def generate_real_samples(X_train, n_samples):
    # choose random instances
    ix = randint(0, X_train.shape[0], n_samples) 
    # retrieve selected images
    X = X_train[ix]  
    # generate class labels
    y = ones((n_samples, 1)) 
    return X, y

# define a function to summarize the performance of the GAN model
def summarize_performance(step, g_model,num_data, latent_dim, n_samples=100,):
    # generate fake samples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # rescale pixel values from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # save the generator model to a file
    filename2 = 'model_data_batch_%04d_%04d_.h5' % (num_data,step+1)
    g_model.save(filename2)
    # copy the saved model to a directory
    import shutil
    shutil.copy(filename2,r'C:\Users\Doaa\Desktop\Final project 2023\Models')
    # print a message indicating that the model has been saved
    print('>Saved: %s' % (filename2))

# define a function to generate and save output images during training
def generate_and_save_output(model,epoch,test_input,num_data):
    # generate images from the model
    predictions = model(test_input,training=False)
    # create a plot of generated images
    fig = plt.figure(figsize=(4,4))
    # plot images
    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow((predictions[i]*127.5+127.5).numpy().astype(np.uint8),cmap='gray')
        plt.axis('off')
    # save the plot to a file
    plt.savefig(f'results_baseline/image_at_epoch_{epoch, num_data}.png')
    plt.show()

# define a function to create a line plot of loss for the GAN model and save it to a file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    # create a subplot for the loss plot
    pyplot.subplot(2, 1, 1)
    # plot the loss of the discriminator on real samples
    pyplot.plot(d1_hist, label='d-real')
    # plot the loss of the discriminator on fake samples
    pyplot.plot(d2_hist, label='d-fake')
    # plot the loss of the generator
    pyplot.plot(g_hist, label='gen')
    # add a legend to the plot
    pyplot.legend()
    # create a subplot for the discriminator accuracy plot
    pyplot.subplot(2, 1, 2)
    # plot the accuracy of the discriminator on real samples
    pyplot.plot(a1_hist, label='acc-real')
    # plot the accuracy of the discriminator on fake samples
    pyplot.plot(a2_hist, label='acc-fake')
    # add a legend to the plot
    pyplot.legend()
    # save therest of the code as an image file and show the plot
    pyplot.savefig('loss_plot.png')
    pyplot.show()

# define the generator model
def define_generator(latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 256 * 7 * 7
    gen = Dense(n_nodes, kernel_initializer=init)(in_lat) # pass the input through a fully connected layer
    gen = LeakyReLU(alpha=0.2)(gen) # apply LeakyReLU activation function to the output of the fully connected layer
    gen = Reshape((7, 7, 256))(gen) # reshape the output to a 7x7x256 feature map
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen) # apply transpose convolution to upsample the feature map to 14x14x128
    gen = LeakyReLU(alpha=0.2)(gen) # apply LeakyReLU activation function to the output of the transpose convolution layer
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen) # apply transpose convolution to upsample the feature map to 28x28x128
    gen = LeakyReLU(alpha=0.2)(gen) # apply LeakyReLU activation function to the output of the transpose convolution layer
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init)(gen) # apply convolution to generate a 28x28x1 output image
    # define the model
    model = Model(in_lat, out_layer)
    return model

# define the discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    # downsample to 14x14
    dis = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image) # apply convolution to downsample the input image to 14x14x64
    dis = LeakyReLU(alpha=0.2)(dis) # apply LeakyReLU activation function to the output of the convolution layer
    dis = Dropout(0.4)(dis) # apply dropout regularization to the output
    # downsample to 7x7
    dis = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(dis) # apply convolution to downsample the feature map to 7x7x64
    dis = LeakyReLU(alpha=0.2)(dis) # apply LeakyReLU activation function to the output of the convolution layer
    dis = Dropout(0.4)(dis) # apply dropout regularization to the output
    # flatten feature maps
    dis = Flatten()(dis) # flatten the feature map to a 1D array
    # real/fake output
    out_classifier = Dense(1, activation='sigmoid')(dis) # apply a fully connected layer with a sigmoid activation function to generate a binary classification output
    # define the model
    model = Model(in_image, out_classifier)
    # compile the model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def custom_loss(alpha):
    # alpha is a hyperparameter that determines the relative weight of the BCE loss compared to the MAE loss
    def loss(y_true, y_pred):
        mae_loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
        bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        return alpha * bce_loss + (1 - alpha) * mae_loss
    return loss

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect the generator output to the discriminator input
    gan_output = d_model(g_model.output)
    # define the gan model as taking latent space vectors and outputting a classification
    model = Model(g_model.input, gan_output)
    # compile the model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=custom_loss(0.5), optimizer=opt)
    return model

from IPython import display

def train(g_model, d_model, gan_model, X_train, latent_dim, num_data, n_epochs=10, n_batch=64):
    # calculate the number of batches per epoch
    bat_per_epo = int(X_train.shape[0] / n_batch)
    # calculate the number of training steps
    n_steps = bat_per_epo * n_epochs
    # initialize lists to hold history of discriminator and generator loss and accuracy
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
    # loop through each epoch
    for epoch in range(n_epochs):
        # loop through each batch in the epoch
        for i in range(bat_per_epo):
            # generate real samples and labels
            X_real, y_real = generate_real_samples(X_train, n_batch)
            # train the discriminator on real samples
            d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real)
            # generate fake samples and labels
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            # train the discriminator on fake samples
            d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake)
            # generate latent points as input for the generator
            z_input = generate_latent_points(latent_dim, n_batch)                 
            # create labels for the generator
            y_gan = ones((n_batch, 1)) 
            # train the generator (via the GAN model) with the discriminator weights fixed
            g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)
        # print the loss and accuracy values for the discriminator and generator
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f], epoch is %d, num of data is %d' % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc,epoch+1,num_data))
        # append the loss and accuracy values to the history lists
        d1_hist.append(d_loss_r)
        d2_hist.append(d_loss_f)
        g_hist.append(g_loss)
        a1_hist.append(d_acc_r)
        a2_hist.append(d_acc_f)
        # every 5 epochs, summarize the generator performance, save the generator and discriminator models to disk, plot the history of the loss and accuracy values, and generate and save sample outputs
        if (epoch+1)%5 == 0:
            summarize_performance(epoch, g_model,num_data, latent_dim)
            save(g_model, d_model, epoch+1, num_data)
            plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist,num_data)
        generate_and_save_output(g_model,epoch+1 ,seed,num_data)

In [ ]:
latent_dim = 100
seed = generate_latent_points(100,16)
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model, data_set, latent_dim)